In [1]:
from feast import FeatureStore
import pandas as pd

In [32]:
entity_df = pd.read_parquet("feast_demo/feature_repo/data/titanic_train_preprocessed.parquet")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,event_timestamp
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2023-03-07 23:36:42.389722
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2023-03-07 23:36:42.389722
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,2023-03-07 23:36:42.389722
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,2023-03-07 23:36:42.389722
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,2023-03-07 23:36:42.389722
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,2023-03-07 23:36:42.389722
179,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S,2023-03-07 23:36:42.389722
180,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,2023-03-07 23:36:42.389722
181,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,2023-03-07 23:36:42.389722


In [31]:
entity_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,event_timestamp
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2023-03-07 23:36:42.389722
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2023-03-07 23:36:42.389722
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,2023-03-07 23:36:42.389722
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,2023-03-07 23:36:42.389722
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,2023-03-07 23:36:42.389722
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,2023-03-07 23:36:42.389722
179,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S,2023-03-07 23:36:42.389722
180,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,2023-03-07 23:36:42.389722
181,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,2023-03-07 23:36:42.389722


In [33]:
pd.read_parquet("feast_demo/feature_repo/data/titanic_pca_feature.parquet")

,PC1,PC2,PC3,PC4,PC5,event_timestamp,PassengerId
0,0.003464,-0.481794,-0.991004,-0.201495,0.050206,2023-03-07 23:36:42.196729,2
1,-0.072348,-0.298333,-1.067847,-0.286986,-0.151978,2023-03-07 23:36:42.196729,4
2,-1.222364,-0.949682,0.258262,0.286256,0.074684,2023-03-07 23:36:42.196729,7
3,0.470111,4.075396,-0.779002,0.583059,0.694532,2023-03-07 23:36:42.196729,11
4,-1.492001,-1.006702,0.155429,0.534607,-0.089904,2023-03-07 23:36:42.196729,12
...,...,...,...,...,...,...,...
178,0.427300,-0.513390,-0.510465,0.967391,-0.557573,2023-03-07 23:36:42.196729,872
179,-1.151445,0.030121,0.053506,-0.530225,-0.636039,2023-03-07 23:36:42.196729,873
180,-0.283813,-0.903174,0.942488,1.004997,-0.194882,2023-03-07 23:36:42.196729,880
181,-0.683439,0.470424,0.149984,-1.227033,-0.617123,2023-03-07 23:36:42.196729,888


In [34]:
feature_store = FeatureStore(repo_path="feast_demo/feature_repo")  # Initialize the feature store

feature_service = feature_store.get_feature_service("titanic_survive_svc_v1")
job = feature_store.get_historical_features(features=feature_service, entity_df=entity_df)

In [35]:
training_df = job.to_df()

In [36]:
print("----- Feature schema -----\n")
print(training_df.info())

print()
print("----- Example features -----\n")
print(training_df.head())

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   PassengerId      183 non-null    int64              
 1   Survived         183 non-null    int64              
 2   Pclass           183 non-null    int64              
 3   Name             183 non-null    object             
 4   Sex              183 non-null    object             
 5   Age              183 non-null    float64            
 6   SibSp            183 non-null    int64              
 7   Parch            183 non-null    int64              
 8   Ticket           183 non-null    object             
 9   Fare             183 non-null    float64            
 10  Cabin            183 non-null    object             
 11  Embarked         183 non-null    object             
 12  event_timestamp  183 non-null    datetime64[ns, UT